In [1]:
# imports
import argparse
from argparse import Namespace

from pytorch_lightning import Trainer, LightningModule, seed_everything
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger, TensorBoardLogger

from yeastdnnexplorer.data_loaders.synthetic_data_loader import SyntheticDataLoader
from yeastdnnexplorer.ml_models.simple_model import SimpleModel

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# set random seed for reproducibility
seed_everything(42)

Seed set to 42


42

In [3]:
# helper function to load a model from a checkpoint file
def load_model(checkpoint_path: str, model: LightningModule = None):
    '''
    Load a model from a checkpoint file
    :param checkpoint_path: path to the checkpoint file (relative to experiments directory)
    :type checkpoint_path: str
    :param model: Class that the model we want to load is an instance of
    :type model: LightningModule
    :return: The model loaded from the checkpoint
    :rtype: LightningModule
    :raises NotImplementedError: If no model is provided
    '''
    if type(model) is not type:
        raise NotImplementedError("Provided model must be an uninstantiated LightningModule class, not an instance of the class.")
    if model is None:
        raise NotImplementedError("No model provided")
    
    # load the model from the checkpoint
    model_checkpoint = model.load_from_checkpoint(checkpoint_path)
    return model_checkpoint

In [4]:
# load in model checkpoint of your choice
# file path is relative to the yeastdnnexplorer/experiemtns directory
model = load_model(
    "logs/tensorboard_logs/lightning_logs/version_39/checkpoints/best-model-epoch=04-val_loss=4.40.ckpt", 
    SimpleModel
)

RuntimeError: Error(s) in loading state_dict for SimpleModel:
	Missing key(s) in state_dict: "linear2.weight", "linear2.bias", "linear3.weight", "linear3.bias". 
	size mismatch for linear1.weight: copying a param with shape torch.Size([10, 10]) from checkpoint, the shape in current model is torch.Size([128, 10]).
	size mismatch for linear1.bias: copying a param with shape torch.Size([10]) from checkpoint, the shape in current model is torch.Size([128]).

Note that when we print out a tensor with all of the path weights between two layers in the neural net, each row of the tensor corresponds to one of the nodes in the output layer. For example, if we have a 10-dim input and a 5-dim output, the first row will have ten entries, and the ith entry in that row will be the path weight from the ith input node to the first output node. (the [i][j] entry will be the path from the jth input to the ith output)

In [15]:
print("Model Hyperparameters===========================================")
print(model.hparams)

print("Model Parameters================================================")
for name, param in model.named_parameters():
    print(f"{name}: {param.size()}")
    print(f"\t{param.data}")

Model Hyperparameters===========================================
"input_dim":  10
"lr":         0.01
"output_dim": 10
Model Parameters================================================
linear1.weight: torch.Size([10, 10])
	tensor([[ 7.9191e-02, -2.0490e-02,  5.8150e-02, -3.1544e-02,  6.9227e-03,
          4.9296e-02,  6.1128e-02,  7.4726e-03,  1.0769e-01,  8.4209e-02],
        [-2.1134e-02, -2.5358e-02, -1.0696e-01, -7.6675e-02,  7.8877e-02,
         -7.9734e-02,  7.7578e-02, -3.1106e-02, -5.7358e-03,  2.2464e-02],
        [ 7.7518e-02, -9.6759e-02, -2.3578e-02,  1.5780e-02,  9.6296e-02,
         -1.1508e-01, -2.6132e-02,  1.1628e-02, -1.7045e-05,  6.7542e-03],
        [ 2.5999e-02, -7.5231e-02,  9.2787e-02, -8.3737e-02, -7.1307e-02,
          8.4179e-02,  4.7821e-02, -1.2645e-02,  5.1505e-02,  1.9591e-02],
        [ 2.6926e-02, -8.4059e-03,  6.9162e-02,  9.5958e-03, -1.3270e-02,
          1.4529e-02,  4.8225e-02,  2.6630e-02, -4.9199e-02,  7.6688e-02],
        [-6.3999e-02, -3.3125e-02,